Helpful Functions pandas

`isna()
fillna()
dropna()
drop()
groupby()
pivot()
pivot_table()
apply()
rolling()
interpolate()
pad()
bfill() ffill()
info()
count()
value_counts()
quantile()`

In [1]:
#import things needed for baseline and data 
#don't worry if there are some things you haven't seen yet, we'll go through it.
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LinearRegression, RidgeCV, Ridge
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Same old Predicting House Prices 💰🏡💰
### Just with missing values

# If anything just use reasoning

In [2]:
#repeat see last notebook for line by line comments explanations of old stuff
data = fetch_openml(name="house_prices", as_frame=True)
ames = data['data']
ames['Price'] = data['target']

In [3]:
#quickly check which columns have lots of missing values
ames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   float64
 1   MSSubClass     1460 non-null   float64
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   float64
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   float64
 18  OverallC

In [4]:
#Fence and Garage have missing values, Fence a lot, Garagve a few, we'll ad some more manually
housing = ames[['Price', 'GrLivArea', 'YearBuilt', 'Fence', 'GarageType']]
X = housing

In [5]:
rng = np.random.default_rng()

In [6]:
X.shape

(1460, 5)

In [7]:
#lets make 20% of GrLivArea np.nan, pretend you didn't see how we made it
nan_indices = rng.choice(X.index.to_list(), size = int(X.shape[0]*0.2))
X.loc[nan_indices, 'GrLivArea'] = np.nan

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [8]:
X.GrLivArea.isna().values.any()

True

In [9]:
X.GarageType.unique()

array(['Attchd', 'Detchd', 'BuiltIn', 'CarPort', None, 'Basment',
       '2Types'], dtype=object)

In [10]:
X.Fence.unique()

array([None, 'MnPrv', 'GdWo', 'GdPrv', 'MnWw'], dtype=object)

# Missing Value Options

In [11]:
#simple
dropped = X.dropna()
dropped.shape

#also simple use a tree based models that deal with it for you
#attention they just naively split on nan and treat it like any other value, 
#trees are powerful, you're handicapping them by just using them this way
#but it could be a quick solution

(216, 5)

yikes, lost almost everything

In [12]:
#look at columns, remember fence is mostly nan, it would be reasonable to drop it, or, make it a feature the model can use
dropFence = X.drop("Fence", axis = 1)
addedNoneFence = X.Fence.apply(lambda x: "None" if x == None else x)

GarageType has a lot of values, have to think more

In [13]:
#what is the distribution?
X.GarageType.value_counts()

Attchd     870
Detchd     387
BuiltIn     88
Basment     19
CarPort      9
2Types       6
Name: GarageType, dtype: int64

# Careful don't do any of the following like I'm showing exactly, need to do it separate for train/test/val sets! 

In [14]:
#most common is attached, then detached, built in, basement (fancy), carport (?), 2types (richie rich)
#reasonable choices are fill with most common, do a groupby and fill with most common for group, fill with none as we did before
#find out about your data a little to make this choice
#semi-reasonable here is to also just drop it, since it isn't a lot of our data (this is relative)
#Another thing is target encoding, but that's tricky to do
#anyway tldr, look and get a feel how this may influence your data
X['GarageType'].fillna('None', inplace = True) #fill for now since groupby ignores otherwise
X.groupby("GarageType").describe()['Price']
#The price is generally lower, if I were a betting man (which I am), I would say it probably means the House has no Garage, but we don't know that
#it's reasonable to leave as is, since it seems to have a significant impact on price
#if you have some features that correlates well with price, I'd groupby that and fill with that, #we don't have that here though
#more advanced use clustering to fill by cluster

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,count,mean,std,min,25%,50%,75%,max
GarageType,,,,,,,,
2Types,6.0,151283.333333,34917.068415,87000.0,146250.0,159000.0,170250.00,186700.0
Attchd,870.0,202892.656322,77146.621778,60000.0,150000.0,185000.0,235000.00,755000.0
Basment,19.0,160570.684211,63967.283574,55993.0,132250.0,148000.0,174000.00,359100.0
BuiltIn,88.0,254751.738636,102230.854394,91000.0,180750.0,227500.0,300719.25,582933.0
CarPort,9.0,109962.111111,24638.183681,82000.0,95000.0,108000.0,110000.00,164900.0
Detchd,387.0,134091.162791,41391.549953,35311.0,110000.0,129500.0,147700.00,475000.0
None,81.0,103317.283951,32815.023389,34900.0,82500.0,100000.0,124000.00,200500.0


In [15]:
#so how about the continuous value we created?
# we can fill with mean, median, max, min, again look at data see what makes sense
X.GrLivArea.describe()

count    1192.000000
mean     1517.243289
std       534.149606
min       334.000000
25%      1143.500000
50%      1457.000000
75%      1774.250000
max      5642.000000
Name: GrLivArea, dtype: float64

In [16]:
#also there is high correlation with GrLivArea so you mgiht want to be careful to not handicap your model
X.corr()

,Price,GrLivArea,YearBuilt
Price,1.000000,0.703382,0.522897
GrLivArea,0.703382,1.000000,0.185506
YearBuilt,0.522897,0.185506,1.000000


In [17]:
#see why
mean_fill = X.copy()
mean_fill['GrLivArea'].fillna(mean_fill.GrLivArea.mean(), inplace = True)
mean_fill.corr()

,Price,GrLivArea,YearBuilt
Price,1.000000,0.643962,0.522897
GrLivArea,0.643962,1.000000,0.167572
YearBuilt,0.522897,0.167572,1.000000


## that had a big impact, could be good, but could also be bad if some nasty outliers. Median probably better.

In [18]:
#can try with Garage Type fill, why? bigger houses are likely to have bigger garages (reasoning!)
X.pivot_table(index = ['GarageType'])

,GrLivArea,Price,YearBuilt
GarageType,,,
2Types,1663.666667,151283.333333,1953.166667
Attchd,1570.035014,202892.656322,1983.009195
Basment,1557.666667,160570.684211,1959.842105
BuiltIn,2140.536232,254751.738636,1995.295455
CarPort,1357.000000,109962.111111,1949.666667
Detchd,1326.145631,134091.162791,1946.777778
None,1191.641791,103317.283951,1942.481481


In [19]:
group_fill = X.copy()
group_fill['GrLivArea'] = group_fill['GrLivArea'].fillna(group_fill.groupby('GarageType')['GrLivArea'].transform('mean'))
group_fill.corr()

,Price,GrLivArea,YearBuilt
Price,1.000000,0.666672,0.522897
GrLivArea,0.666672,1.000000,0.206792
YearBuilt,0.522897,0.206792,1.000000


### looks like it's lowering the correlation too but not as much. 

In [20]:
group_fill.pivot_table(index = ['GarageType'])
#within group means stay the same of course since we just filled with those

,GrLivArea,Price,YearBuilt
GarageType,,,
2Types,1663.666667,151283.333333,1953.166667
Attchd,1570.035014,202892.656322,1983.009195
Basment,1557.666667,160570.684211,1959.842105
BuiltIn,2140.536232,254751.738636,1995.295455
CarPort,1357.000000,109962.111111,1949.666667
Detchd,1326.145631,134091.162791,1946.777778
None,1191.641791,103317.283951,1942.481481


In [21]:
#so same method but often a little better
scaler = StandardScaler()
scaled_X = X.copy()
scaled_X['GrLivArea'] = scaler.fit_transform(scaled_X['GrLivArea'].to_numpy().reshape(-1, 1))
scaled_X.fillna(0)

,Price,GrLivArea,YearBuilt,Fence,GarageType
0,208500.0,0.361018,2003.0,0,Attchd
1,181500.0,-0.478050,1976.0,0,Attchd
2,223500.0,0.503360,2001.0,0,Attchd
3,140000.0,0.374128,1915.0,0,Detchd
4,250000.0,1.275003,2000.0,0,Attchd
...,...,...,...,...,...
1455,175000.0,0.243024,1999.0,0,Attchd
1456,210000.0,1.040888,1978.0,MnPrv,Attchd
1457,266500.0,1.540958,1941.0,GdPrv,Attchd
1458,142125.0,-0.822668,1950.0,0,Attchd


### scale and standardize, then fill with 0, this is still the mean but it basically tells the model it's unimportant

In [22]:
#even better, tell the model you filled nans, don't loose info !
scaler = StandardScaler()
scaled_X = X.copy()
scaled_X['LivArea_was_nan'] = scaled_X['GrLivArea'].isna().apply(int)
scaled_X['GrLivArea'] = scaler.fit_transform(scaled_X['GrLivArea'].to_numpy().reshape(-1, 1))
scaled_X.GrLivArea.fillna(0, inplace = True)

In [23]:
X

,Price,GrLivArea,YearBuilt,Fence,GarageType
0,208500.0,1710.0,2003.0,None,Attchd
1,181500.0,1262.0,1976.0,None,Attchd
2,223500.0,1786.0,2001.0,None,Attchd
3,140000.0,1717.0,1915.0,None,Detchd
4,250000.0,2198.0,2000.0,None,Attchd
...,...,...,...,...,...
1455,175000.0,1647.0,1999.0,None,Attchd
1456,210000.0,2073.0,1978.0,MnPrv,Attchd
1457,266500.0,2340.0,1941.0,GdPrv,Attchd
1458,142125.0,1078.0,1950.0,None,Attchd


# Correct way
## very important, don't leak info to your test set, always do everything on train set, then on your test set
## keep these separate or your CV could not be accurate, you could overfit

In [24]:
from lightgbm import LGBMRegressor
X_original = X.copy()
X["Fence"] = X.Fence.apply(lambda x: "None" if x == None else x)
X["GarageType"] = X.GarageType.apply(lambda x: "None" if x == None else x)
encoder = LabelEncoder() #for Trees, think about what you're doing, not right for every model
X["Fence"] = encoder.fit_transform(X["Fence"].to_numpy().reshape(-1, 1))
encoder = LabelEncoder() #for Trees, think about what you're doing, not right for every model
X["GarageType"] = encoder.fit_transform(X["GarageType"].to_numpy().reshape(-1, 1))


# example 1 (depends on your strategy, other ways are ok too, just don't leak, think about leaking info)
n_folds = 5
#this lets you create 5 folds on which you can train and validate on you import it from sklearn.model_selection
kfold = KFold(n_splits = n_folds, random_state = 42)
y = X.pop("Price")
for fold, (train_indices, validation_indices) in enumerate(kfold.split(X)): #note we still have our test set
    model = LGBMRegressor()
    #train on 4 folds
    X_train, X_test, y_train, y_test = X.loc[train_indices, :], X.loc[validation_indices, :], y.iloc[train_indices], y.iloc[validation_indices]
    
    #get mean of train set
    train_mean = X.GrLivArea.mean()
    #let model know where you filled in values
    X_train['LivArea_was_nan'] = X_train['GrLivArea'].isna().apply(int)
    #fill train set with mean of train set
    X_train['GrLivArea'].fillna(train_mean, inplace = True)
    
    #let model know where you filled nans in test
    X_test['LivArea_was_nan'] = X_test['GrLivArea'].isna().apply(int)
    #fill Test set with Mean of TRAIN set
    X_test.GrLivArea.fillna(train_mean, inplace = True)
    model.fit(X_train, y_train)
    #check performance on 5th fold
    val_pred = model.predict(X_test)
    val_score = mse(val_pred, y_test)
    print(f"score on fold {fold} was {val_score}")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the sha

score on fold 0 was 1579651661.1964133
score on fold 1 was 2552377717.634608
score on fold 2 was 1971916707.1744025
score on fold 3 was 1619503982.4125078
score on fold 4 was 2841374857.949645


In [25]:
from lightgbm import LGBMRegressor
X_original = X.copy()
X["Fence"] = X.Fence.apply(lambda x: "None" if x == None else x)
X["GarageType"] = X.GarageType.apply(lambda x: "None" if x == None else x)
encoder = LabelEncoder() #for Trees, think about what you're doing, not right for every model
X["Fence"] = encoder.fit_transform(X["Fence"].to_numpy().reshape(-1, 1))
encoder = LabelEncoder() #for Trees, think about what you're doing, not right for every model
X["GarageType"] = encoder.fit_transform(X["GarageType"].to_numpy().reshape(-1, 1))


# example 2 (depends on your strategy, other ways are ok too, just don't leak, think about leaking info)
n_folds = 5
#this lets you create 5 folds on which you can train and validate on you import it from sklearn.model_selection
kfold = KFold(n_splits = n_folds, random_state = 42)
for fold, (train_indices, validation_indices) in enumerate(kfold.split(X)): #note we still have our test set
    model = LGBMRegressor()
    #train on 4 folds
    X_train, X_test, y_train, y_test = X.loc[train_indices, :], X.loc[validation_indices, :], y.iloc[train_indices], y.iloc[validation_indices]
    
    
    scaler = StandardScaler()
    X_train['LivArea_was_nan'] = X_train['GrLivArea'].isna().apply(int)
    #scaler is fit on Train Set only! Not the whole!
    X_train['GrLivArea'] = scaler.fit_transform(X_train['GrLivArea'].to_numpy().reshape(-1, 1))
    X_train.GrLivArea.fillna(0, inplace = True)
    
    #repeat for test set
    #scaler transforms Test Set based on Train Set distribution, not the whole!
    X_test['LivArea_was_nan'] = X_test['GrLivArea'].isna().apply(int)
    X_test['GrLivArea'] = scaler.transform(X_test['GrLivArea'].to_numpy().reshape(-1, 1))
    X_test.GrLivArea.fillna(0, inplace = True)
    model.fit(X_train, y_train)
    #check performance on 5th fold
    val_pred = model.predict(X_test)
    val_score = mse(val_pred, y_test)
    print(f"score on fold {fold} was {val_score}")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the sha

score on fold 0 was 1592171856.7008302
score on fold 1 was 2556107352.609457
score on fold 2 was 1968004041.1246755
score on fold 3 was 1649711549.2210736
score on fold 4 was 2788719092.759506


# not universal!
## think about your data before filling in values, what would make sense
## can you substitute? Check wikipedia sometimes to get some better understanding it's 15 minutes of reading saves 2 hours sometimes